<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/crypto_price_predictn_with_ind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=6ae0f8bc0bbc56bdfd5ee489dd1ef2446d91ec34db12cc41ce06d367bcec8ae5
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [15]:
import pandas as pd
import yfinance as yf
from ta.momentum import StochasticOscillator, RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [16]:
import pandas as pd
import yfinance as yf
from ta.momentum import StochasticOscillator, RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

def fetch_and_preprocess_data(tickers, start, end):
    # Fetch historical data
    data = yf.download(tickers, start=start, end=end)

    # Prepare an empty DataFrame to store results
    result = pd.DataFrame()

    for ticker in tickers:
        # Extract high, low, and close prices
        high = data['High'][ticker]
        low = data['Low'][ticker]
        close = data['Close'][ticker]

        # Compute Stochastic RSI
        stoch_rsi = StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3, fillna=True)
        result[f'{ticker}_stoch_rsi'] = stoch_rsi.stoch()

        # Compute RSI
        rsi = RSIIndicator(close=close, window=14, fillna=True)
        result[f'{ticker}_rsi'] = rsi.rsi()

        # Compute Bollinger Bands
        bb = BollingerBands(close=close, window=20, window_dev=2, fillna=True)
        result[f'{ticker}_bb_high'] = bb.bollinger_hband()
        result[f'{ticker}_bb_low'] = bb.bollinger_lband()
        result[f'{ticker}_bb_mid'] = bb.bollinger_mavg()

        # Compute EMA Cross
        ema_short = EMAIndicator(close=close, window=12, fillna=True).ema_indicator()
        ema_long = EMAIndicator(close=close, window=26, fillna=True).ema_indicator()
        result[f'{ticker}_ema_cross'] = ema_short - ema_long

    return result

tickers = ["BTC-USD", "ETH-USD", "LTC-USD"]
start_date = "2020-01-01"
end_date = "2024-07-30"
data = fetch_and_preprocess_data(tickers, start=start_date, end=end_date)


[*********************100%%**********************]  3 of 3 completed


In [17]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        x_data = data.iloc[i:(i + time_step), :].values
        X.append(x_data)
        y.append(data.iloc[i + time_step, 0])  # Predicting the next closing price of the first ticker
    return np.array(X), np.array(y)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Create the dataset
time_step = 60
X, y = create_dataset(pd.DataFrame(data_scaled), time_step)

# Reshape input to be [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])


In [19]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, X.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)
model.fit(X, y, batch_size=64, epochs=100, verbose=1, callbacks=[early_stop])


Epoch 1/100
26/26 [==============================] - 6s 69ms/step - loss: 0.0828
Epoch 2/100
26/26 [==============================] - 2s 70ms/step - loss: 0.0567
Epoch 3/100
26/26 [==============================] - 2s 70ms/step - loss: 0.0462
Epoch 4/100
26/26 [==============================] - 2s 95ms/step - loss: 0.0405
Epoch 5/100
26/26 [==============================] - 3s 99ms/step - loss: 0.0366
Epoch 6/100
26/26 [==============================] - 2s 70ms/step - loss: 0.0328
Epoch 7/100
26/26 [==============================] - 2s 69ms/step - loss: 0.0338
Epoch 8/100
26/26 [==============================] - 2s 68ms/step - loss: 0.0319
Epoch 9/100
26/26 [==============================] - 2s 70ms/step - loss: 0.0277
Epoch 10/100
26/26 [==============================] - 2s 70ms/step - loss: 0.0266
Epoch 11/100
26/26 [==============================] - 3s 107ms/step - loss: 0.0272
Epoch 12/100
26/26 [==============================] - 2s 84ms/step - loss: 0.0259
Epoch 13/100
26/26 [====

In [20]:
import time

def make_trading_decision(model, scaler, tickers):
    while True:
        # Fetch the latest data
        new_data = fetch_and_preprocess_data(tickers, start=start_date, end=end_date).iloc[-time_step:]
        new_data_scaled = scaler.transform(new_data)

        # Prepare the input data
        x = new_data_scaled.reshape(1, time_step, new_data_scaled.shape[1])

        # Make prediction
        predicted_price = model.predict(x)
        predicted_price = scaler.inverse_transform(np.hstack([predicted_price, np.zeros((predicted_price.shape[0], new_data_scaled.shape[1]-1))]))[:, 0]

        # Implement your trading logic here
        print(f"Predicted price for {tickers[0]}: {predicted_price[0]}")

        # Sleep for a minute before the next prediction
        time.sleep(60)

# Start the trading bot
make_trading_decision(model, scaler, tickers)


[*********************100%%**********************]  3 of 3 completed


1/1 [==============================] - 1s 855ms/step
Predicted price for BTC-USD: 61.569589376449585


[*********************100%%**********************]  3 of 3 completed


1/1 [==============================] - 0s 43ms/step
Predicted price for BTC-USD: 61.62627935409546


[*********************100%%**********************]  3 of 3 completed


...

[*********************100%%**********************]  3 of 3 completed


1/1 [==============================] - 0s 33ms/step
Predicted price for BTC-USD: 60.07172465324402


[*********************100%%**********************]  3 of 3 completed


1/1 [==============================] - 0s 50ms/step
Predicted price for BTC-USD: 60.0847065448761


KeyboardInterrupt: 